In [35]:
import os
import sys
from dataclasses import dataclass
from pathlib import Path
from cnnClassifier import logging,CustomException
import urllib.request as request
import zipfile

THIS JUPYTER NOTEBOOK WILL RUN BEDEFAULT IN THE RESEARCH FOLDER HENCE WE NEED TO CHANGE OUR DIRECTORY ONE LEVEL UP

In [40]:
# os.chdir('D:\\codes\\DeepLearning_Proj\\proj1\\research')
curr_working_dir = os.getcwd()
print(curr_working_dir)

D:\codes\DeepLearning_Proj\proj1\research


RUN THE BELOW CODE IF YOU HAVE MOVED COUPLE OF DIRECTORY ABOVE TO COME BACK TO THE ORIGINAL DIRECTORY

In [41]:
# os.chdir(curr_working_dir) 

In [42]:
os.chdir('../')  # MOVE ONE LEVEL UP
%pwd

'D:\\codes\\DeepLearning_Proj\\proj1'

In [43]:
@dataclass(frozen = True)
class DataIngestionConfig:  #(BELOW ARE THE RETURN TYPES OF THE FUNCTION)
    root_dir         : Path
    source_URL       : str
    local_data_file  : Path
    unzip_dir        : Path

WE ARE IMPORTING EVERY THING FROM THE CONSTANT FILE BECAUSE IT CONTAINS THE FILE PATH WHERE THE LINKS TO USE THE DATASET IS AVAILABLE

In [44]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directory,get_size

In [118]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
    
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])  # THIS WILL CREATE THE PARENT DIRECTORY AS (artifacts)

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directory([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir         = config.root_dir,
            source_URL       = config.source_URL,
            local_data_file  = config.local_data_file,
            unzip_dir        = config.unzip_dir
        )

        return data_ingestion_config

request.urlretrieve -> THIS LINE CALLS THE 'urlretrieve' FROM THE 'urllib.request' MODULE. 'urlretrieve' IS USED TO DOWNLOAD A FILE FROM A REMOTE URL AND SAVE IT LOCALLY.

IT WILL RETURN 2 VALUES 

1) filename -> the local file path where the downloaded content is saved 

2) header -> contain the HTTP header of the response from the remote server

In [140]:
class DataIngestion:
    def __init__(self, config : DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url = self.config.source_URL,             # THE LINK OR THE SOURCE FROM WHERE THE FILE HAS TO BE DOWNLOADED 
                filename = self.config.local_data_file    # THE LOCAL PATH WHERE THE FILE HAS TO BE SAVED
            )
            logging.info(f'{filename} DOWNLOADED WITH THE FOLOWING INFO {header}')
        else:
            logging.info(f'file already exist of size : {get_size(Path(self.config.local_data_file))}')

    def extract_zip_file(self):
        '''
            zip_file_path : str
            extract the zip file into the data diretory 
            function returns none
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [141]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    CustomException(e,sys)

[2023-10-25 15:55:28,603] 31 root - INFO - YAML FILE config\config.yaml LOADED SUCCESSFULLY
[2023-10-25 15:55:28,605] 31 root - INFO - YAML FILE params.yaml LOADED SUCCESSFULLY
[2023-10-25 15:55:28,606] 50 root - INFO - CREATED DIRECTORY AT : artifacts
[2023-10-25 15:55:28,607] 50 root - INFO - CREATED DIRECTORY AT : artifacts/data_ingestion
[2023-10-25 15:55:32,191] 11 root - INFO - artifacts/data_ingestion/data.zip DOWNLOADED WITH THE FOLOWING INFO Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f628ef17787ed48d610db016ad6dead1490f6a028d10cc6a9c52fb5e57cba1b9"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 489A:3C6C:3ED879:4CB5B6:6538E73D
Accept-Ranges: bytes
Date: Wed, 25 Oct 2023 10:25:31 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10226-MAA
X-